In [59]:
import riskfolio as rp
import pandas as pd
import numpy as np
import yfinance as yf
import warnings
import xlwings as xw
import os
import matplotlib.pyplot as plt


In [66]:
# Function to calculate portfolio performance and save reports using Riskfolio-Lib plots

# Create the directory for reports if it doesn't exist
report_dir = "../reports/05 - portfolio_optimization_riskfolio"
os.makedirs(report_dir, exist_ok=True)


# Example of running the repor

In [67]:

warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.4%}'.format

# Load S&P 500 ticker symbols
#sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
tickers = ['SMCI', 'NVDA', 'HRB', 'CL', 'XPO', 'RCL', 'HD', 'EXAS', 'BLDR', 'FTNT',
           'ORCL', 'GE', 'TPX', 'CCL', 'LII', 'SAIA', 'FTI', 'PHM', 'META', 'CELH',
           'LRCX', 'SPB', 'JBL', 'AVGO', 'ADBE', 'VRSK', 'SSD', 'LSCC', 'EME',
           'TOL', 'CXT', 'HUBS', 'APO', 'AMAT', 'MSI', 'MA', 'FIX', 'SCCO', 'ALSN',
           'KLAC', 'NFLX', 'CAT', 'OC', 'PVH', 'LLY', 'AXON', 'EXP', 'LNW', 'URI',
           'KMB']

# Download historical price data
data = yf.download(tickers, start='2024-01-01', end='2024-09-30', interval='1h')['Adj Close']
data = data.tz_localize(None)

# Forward-fill missing values and drop any columns that still have NaNs
data = data.ffill().dropna(axis=1)

# Calculate daily returns from adjusted closing prices
returns = data.pct_change().dropna()

# Display the first few rows of returns to verify the data is ready for portfolio optimization
returns.head()


[*********************100%***********************]  50 of 50 completed


Ticker,ADBE,ALSN,AMAT,APO,AVGO,AXON,BLDR,CAT,CCL,CELH,...,SAIA,SCCO,SMCI,SPB,SSD,TOL,TPX,URI,VRSK,XPO
Datetime,,,,,,,,,,,,,,,,,,,,,
2024-01-02 15:30:00,0.0190%,-1.3260%,0.3647%,-0.0793%,-0.1135%,0.5256%,-0.0723%,-0.5253%,-1.3955%,3.1046%,...,-1.2443%,-0.0235%,1.0932%,-0.0811%,-0.7086%,-0.6714%,-0.4184%,-0.5990%,-0.2494%,0.0000%
2024-01-02 16:30:00,-0.2034%,0.1309%,-0.1785%,0.5367%,-0.1611%,0.1782%,-0.0784%,-0.0409%,0.4246%,0.2080%,...,-0.4827%,-0.2630%,0.0246%,0.3373%,-0.1603%,0.0641%,0.1100%,0.2029%,-0.1288%,-0.3410%
2024-01-02 17:30:00,0.0078%,-0.3050%,-0.1501%,-0.1961%,-0.2533%,-0.4965%,-0.1749%,0.1465%,-0.4515%,0.0173%,...,-0.3582%,-0.3193%,0.1860%,0.4357%,-0.2797%,-0.1972%,-0.3398%,0.0581%,-0.0969%,-0.3775%
2024-01-02 18:30:00,0.3100%,-0.3759%,-0.3614%,-0.2947%,-0.3580%,-0.4308%,-0.3444%,-0.3369%,-1.4434%,0.7096%,...,-0.1049%,-0.1029%,-0.2925%,-0.0248%,-0.0987%,-0.4249%,0.0501%,-0.3168%,0.2384%,0.1776%
2024-01-02 19:30:00,-0.4476%,-0.2896%,-1.3287%,-0.2737%,-0.5203%,-0.1297%,-0.3275%,-0.4678%,-0.3735%,0.3093%,...,0.8625%,-0.9415%,-0.4129%,-0.6385%,0.1612%,-0.2977%,-0.5312%,-0.4873%,-0.2041%,-0.1891%


## Markowitz Mean-Variance Analysis Framework

### Introduction

The **Mean-Variance Analysis**, introduced by Harry Markowitz in 1952, is the foundation of modern portfolio theory (MPT). This framework provides a method for constructing a portfolio that maximizes expected return for a given level of risk or minimizes risk for a given level of expected return. It is based on two key parameters of asset returns:
1. **Expected Return (Mean)**: The anticipated average return of an asset or portfolio.
2. **Risk (Variance or Standard Deviation)**: The degree of variation in asset returns, measured by the variance or its square root, the standard deviation.

Markowitz's model assumes that investors are risk-averse, meaning they prefer less risk for the same level of expected return. As a result, the Mean-Variance Optimization aims to find the portfolio that provides the best possible trade-off between risk and return.

### Key Components

1. **Expected Return**: The expected return of a portfolio is the weighted sum of the expected returns of the individual assets. Mathematically:
    $$
    E(R_p) = \sum_{i=1}^{n} w_i \cdot E(R_i)
    $$
    
-   Where:
    - $E(R_p)$ = Expected return of the portfolio
    - $w_i$ = Weight of asset $i$ in the portfolio
    - $E(R_i)$ = Expected return of asset $i$

2. **Portfolio Variance (Risk)**: The portfolio variance captures the total risk of the portfolio, accounting for the variances of individual assets and the covariances between them. It is calculated as:
$$
\text{Var}(R_p) = \sum_{i=1}^{n} \sum_{j=1}^{n} w_i \cdot w_j \cdot \text{Cov}(R_i, R_j)
$$
-   Where:
    - $\text{Var}(R_p)$ = Variance of the portfolio
    - $w_i, w_j$ = Weights of assets $i$ and $j$
    - $\text{Cov}(R_i, R_j)$ = Covariance between the returns of assets $i$ and $j$
- The **standard deviation** is the square root of the variance and is commonly used as a measure of risk.

3. **Covariance Matrix**:
The covariance matrix plays a central role in determining portfolio risk. It measures how asset returns move together (correlate). Positive covariance implies that two assets tend to move in the same direction, while negative covariance implies that they move in opposite directions.
   
4. **Efficient Frontier**:
The efficient frontier is a curve that represents the set of optimal portfolios that offer the maximum expected return for a given level of risk or the minimum risk for a given expected return. Portfolios on this frontier are considered "efficient."
   
5. **Global Minimum Variance Portfolio**:
This is the portfolio on the efficient frontier that has the lowest possible risk (variance). It is often used by very risk-averse investors.
   
6. **Capital Allocation Line (CAL)**:
When we include a risk-free asset (like government bonds), the Capital Allocation Line represents the risk-return combinations of portfolios formed by blending the risk-free asset and risky portfolios. The point where the CAL is tangent to the efficient frontier gives the **tangency portfolio**, which has the highest Sharpe ratio (risk-adjusted return).

### Assumptions of Mean-Variance Optimization
1. **Risk and Return are normally distributed**: The model assumes that asset returns follow a normal distribution, which allows risk to be summarized using variance.
2. **Investors are rational and risk-averse**: Investors prefer less risk for a given level of expected return.
3. **Investors make decisions based on expected return and variance only**: Other factors, such as liquidity or transaction costs, are not considered.
4. **Markets are efficient**: Asset prices reflect all available information.

### Optimization Problem

The Mean-Variance Optimization involves solving a quadratic programming problem where the objective is to minimize portfolio variance, subject to constraints on the expected return and portfolio weights. The general optimization problem can be formulated as:

$$
\min_{w} \quad w^\top \Sigma w
$$
$$
\text{subject to} \quad w^\top \mu = E(R_p), \quad \sum w_i = 1, \quad w_i \geq 0 \, \forall i
$$

Where:
- $w$ = Portfolio weights vector.
- $\Sigma$ = Covariance matrix of asset returns.
- $\mu$ = Vector of expected returns of assets.
- $E(R_p)$ = Expected return of the portfolio.

The constraint $\sum w_i = 1$ ensures that the portfolio weights sum to 100%, and $w_i \geq 0$ ensures that short selling (negative weights) is not allowed.

### Benefits of Mean-Variance Optimization
- Provides a structured approach to portfolio selection.
- Helps investors achieve an optimal trade-off between risk and return.
- Supports diversification by explicitly accounting for asset correlations (via the covariance matrix).

### Limitations of Mean-Variance Optimization
- **Estimation Errors**: The optimization is highly sensitive to the inputs (expected returns and covariances). Small changes in these estimates can result in significantly different portfolio weights.
- **Assumption of Normality**: Asset returns may not always follow a normal distribution, especially in the case of fat tails or skewed distributions.
- **Ignores Higher Moments**: The model focuses only on the first two moments (mean and variance), neglecting skewness and kurtosis.
- **Doesn't Consider Transaction Costs**: The model doesn't account for real-world costs such as transaction fees or taxes when rebalancing the portfolio.

### Conclusion

Markowitz’s Mean-Variance framework is a powerful tool for portfolio optimization, and it laid the foundation for modern portfolio theory. However, while it offers valuable insights into the risk-return trade-off, its real-world application requires careful handling of input estimates and may need to be complemented with more advanced models that consider factors beyond mean and variance.



In [89]:
# Initialize a dictionary to store portfolio weights for each optimization method
portfolio_weights = {}

# Create the directory for reports if it doesn't exist
report_dir = "../reports/05 - portfolio_optimization_riskfolio"
os.makedirs(report_dir, exist_ok=True)

# Refactored function to handle various portfolio optimization methods
def portfolio_optimization(pf_returns, method_num, method_name, **kwargs):
    method_id = f"{method_num:02d} - {method_name}"
    file_path = os.path.join(report_dir, f"{method_id}")

    print(f"Running {method_name} Portfolio Optimization")

    # Building the portfolio object
    port = rp.Portfolio(returns=pf_returns)

    # Estimate input parameters for optimization
    method_mu = kwargs.get('method_mu', 'hist')  # Default is historical mean returns
    method_cov = kwargs.get('method_cov', 'hist')  # Default is historical covariance

    # Get the asset stats based on the specified method for expected returns and covariance
    port.assets_stats(method_mu=method_mu, method_cov=method_cov)

    # Optimization method parameters
    model = kwargs.get('model', 'Classic')  # Default model is 'Classic'
    rm = kwargs.get('rm', 'MV')  # Default risk measure is Mean-Variance
    obj = kwargs.get('obj', 'Sharpe')  # Default objective is 'Sharpe'
    rf = kwargs.get('rf', 0)  # Default risk-free rate is 0
    l = kwargs.get('l', 0)  # Risk aversion factor, useful for Utility maximization
    hist = kwargs.get('hist', True)  # Use historical scenarios by default

    # Perform the optimization
    #w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
    w = port.owa_optimization(obj='Sharpe', owa_w=None, kelly=False, rf=0, l=2)

    # Store the portfolio weights
    portfolio_weights = {method_name: w}  # Store the weights for later use

    # Efficient frontier generation
    points = kwargs.get('points', 50)  # Number of points on the efficient frontier
    frontier = port.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

    # Plotting and Reporting using Riskfolio-Lib
    label = kwargs.get('label', 'Max Risk Adjusted Return Portfolio')
    mu = port.mu  # Expected returns
    cov = port.cov  # Covariance matrix
    returns = port.returns  # Returns of the assets

    # Excel Report Generation
    rp.excel_report(returns, w, rf=rf, alpha=0.05, t_factor=252, ini_days=1, days_per_year=252, name=file_path)

    # Generate and save plots
    fig, ax = plt.subplots(figsize=(10, 10))
    ax = rp.plot_drawdown(returns, w, alpha=0.05, height=10, width=10, ax=ax)

    fig_1, ax_1 = plt.subplots(figsize=(10, 5))
    ax_1 = rp.plot_pie(w=w, title=f'{method_name} Weights', others=0.05, nrow=25, cmap="tab20", height=5, width=10, ax=ax_1)

    fig_2, ax_2 = plt.subplots(figsize=(10, 5))
    ax_2 = rp.plot_risk_con(w=w, cov=cov, returns=returns, rm=rm, rf=rf, alpha=0.05, t_factor=252, height=5, width=10, ax=ax_2)

    fig_3, ax_3 = plt.subplots(figsize=(10, 5))
    ax_3 = rp.plot_bar(w, title='Portfolio Weights', kind="v", others=0.05, nrow=25, height=5, width=10, ax=ax_3)

    fig_4, ax_4 = plt.subplots(figsize=(10, 5))
    ax_4 = rp.plot_hist(returns, w, alpha=0.05, bins=50, height=5, width=10, ax=ax_4)

    fig_5, ax_5 = plt.subplots(figsize=(10, 5))
    ax_5 = rp.plot_frontier(w_frontier=frontier, mu=mu, cov=cov, returns=returns, rm=rm, rf=rf, alpha=0.01, cmap='viridis', w=w, label=label, marker='*', s=16, c='r', height=5, width=10, ax=ax_5)

    # Save the results to Excel
    with xw.App(visible=False) as app:
        wb = xw.Book(f"{file_path}.xlsx")
        sheet1 = wb.sheets[0]

        # Add pictures of the plots to the Excel sheet
        sheet1.pictures.add(fig, name="Drawdown Plot", update=True, top=sheet1.range("O1").top, left=sheet1.range("O1").left)
        sheet1.pictures.add(fig_1, name="Pie Chart of Weights", update=True, top=sheet1.range("D38").top, left=sheet1.range("D38").left)
        sheet1.pictures.add(fig_2, name="Risk Contribution", update=True, top=sheet1.range("O38").top, left=sheet1.range("O38").left)
        sheet1.pictures.add(fig_3, name="Weights Bar Plot", update=True, top=sheet1.range("D65").top, left=sheet1.range("D65").left)
        sheet1.pictures.add(fig_4, name="Histogram of Returns", update=True, top=sheet1.range("O65").top, left=sheet1.range("O65").left)
        sheet1.pictures.add(fig_5, name="Efficient Frontier", update=True, top=sheet1.range("D9").top, left=sheet1.range("D9").left)

        # Save the Excel file
        wb.save(f"{file_path}.xlsx")
        wb.close()

    return w


# Mean Risk Portfolio Optimization using historical estimates
portfolio_optimization(
    pf_returns=returns,
    method_num=1,
    method_name='Mean Risk Portfolio Optimization - OWA_Optimization',
    method_mu='hist',
    method_cov='hist',
    model='Classic',
    rm='MV',
    obj='Sharpe',
    rf=0,
    l=0,
    hist=True
)

# Example of running the report creation for each method
#create_report(1, 'mean_risk_historical', returns, w_mean_risk)



Running Mean Risk Portfolio Optimization - OWA_Optimization Portfolio Optimization


,weights
ADBE,0.0000%
ALSN,11.4270%
AMAT,0.0000%
APO,0.0000%
AVGO,0.0000%
AXON,0.0000%
BLDR,0.0000%
CAT,0.0000%
CCL,0.0000%
CELH,0.0000%


## Mean Risk Portfolio Optimization using Custom Estimates (Mean and Covariance)

In some cases, using historical estimates for mean returns and the covariance matrix may not provide the best predictions for future returns and risks. Instead, you might want to use custom estimates based on other models, expert opinions, or forecasts. This approach allows you to input your own expected returns and covariance matrix for portfolio optimization.

### Key Concepts:
- **Custom Mean Returns**: Expected returns provided by the user, instead of relying on historical data.
- **Custom Covariance Matrix**: Covariance of assets provided by the user, which might come from advanced models or external forecasts.
- **Mean-Variance Optimization**: Still using the same mean-variance optimization framework but with custom inputs.


In [51]:
# 2. Mean-Risk Portfolio Optimization using custom estimates
def mean_risk_custom_optimization(pf_returns):
    print("Running Mean-Risk Portfolio Optimization using custom estimates")
    # Example of custom mean and covariance matrix (replace with actual data for real case)
    custom_mean = pd.Series(np.random.rand(pf_returns.shape[1]), index=pf_returns.columns)
    custom_cov = pd.DataFrame(np.random.rand(pf_returns.shape[1], pf_returns.shape[1]), index=pf_returns.columns, columns=pf_returns.columns)
    port = rp.Portfolio(returns=pf_returns)
    port.mu = custom_mean
    port.cov = custom_cov
    weights = port.optimization(model='Classic', rm='MV')  # Mean-Variance optimization
    portfolio_weights['mean_risk_custom'] = weights  # Store weights
    return weights


In [52]:
# Run optimizations and store results
w_mean_risk = mean_risk_optimization(returns)
w_custom = mean_risk_custom_optimization(returns)

# Print stored weights
print("Stored Portfolio Weights:")
for method, weights in portfolio_weights.items():
    print(f"{method}: {weights}")


Running Mean-Risk Portfolio Optimization using historical estimates
Running Mean-Risk Portfolio Optimization using custom estimates
Stored Portfolio Weights:
mean_risk_historical:       weights
ADBE  0.0000%
ALSN  6.4503%
AMAT  0.0000%
APO   0.0000%
AVGO  0.0000%
AXON  0.0000%
BLDR  0.0000%
CAT   0.0000%
CCL   0.0000%
CELH  0.0000%
CL   39.2248%
CXT   0.0000%
EME   9.6842%
EXAS  0.0000%
EXP   0.0000%
FIX   0.0000%
FTI   0.0000%
FTNT  0.0000%
GE    4.6470%
HD    0.0000%
HRB   1.1978%
HUBS  0.0000%
JBL   0.0000%
KLAC  0.0000%
KMB   0.2297%
LII   0.0000%
LLY   3.2050%
LNW   0.0000%
LRCX  0.0000%
LSCC  0.0000%
MA    0.0000%
META  1.8467%
MSI  20.3368%
NFLX  2.5206%
NVDA  5.9739%
OC    0.0000%
ORCL  3.7002%
PHM   0.0000%
PVH   0.0000%
RCL   0.0000%
SAIA  0.0000%
SCCO  0.9828%
SMCI  0.0000%
SPB   0.0000%
SSD   0.0000%
TOL   0.0000%
TPX   0.0000%
URI   0.0000%
VRSK  0.0000%
XPO   0.0000%
mean_risk_custom:       weights
ADBE  0.0000%
ALSN  0.0000%
AMAT  6.3486%
APO   0.0000%
AVGO  0.0000%
AXON

In [53]:


# 2. Mean-Risk with Custom Estimates
def mean_risk_custom_optimization(returns, custom_mean, custom_cov):
    print("Running Mean-Risk Portfolio Optimization using custom estimates")
    return optimize_portfolio(returns, model='Classic', risk_measure='MV', custom_mean=custom_mean, custom_cov=custom_cov)

# 3. Ulcer Index Portfolio Optimization
def ulcer_index_optimization(returns):
    print("Running Ulcer Index Portfolio Optimization")
    return optimize_portfolio(returns, model='Classic', risk_measure='UI')

# 4. Entropic Value at Risk (EVaR) Optimization
def evar_optimization(returns):
    print("Running EVaR Portfolio Optimization")
    return optimize_portfolio(returns, model='Classic', risk_measure='EVaR')

# 5. Gini Mean Difference (GMD) Optimization
def gmd_optimization(returns):
    print("Running GMD Portfolio Optimization")
    return optimize_portfolio(returns, model='Classic', risk_measure='GMD')

# 6. Black-Litterman Model
def black_litterman_optimization(returns, P, Q, Omega):
    print("Running Black-Litterman Model Portfolio Optimization")
    port = rp.Portfolio(returns=returns)
    port.mu_bl, port.cov_bl = port.black_litterman(P, Q, Omega)
    w = port.optimization(model='BL')
    return w

# 7. Risk Parity Optimization
def risk_parity_optimization(returns):
    print("Running Risk Parity Optimization")
    return optimize_portfolio(returns, model='RiskParity', risk_measure='MV')

# 8. Hierarchical Risk Parity (HRP) Optimization
def hrp_optimization(returns):
    print("Running HRP Portfolio Optimization")
    port = rp.HCPortfolio(returns=returns)
    w = port.optimization(model='HRP', codependence='pearson', linkage='single', risk_measure='MV')
    return w

# 9. Hierarchical Clustering and Networks
def hierarchical_clustering_optimization(returns):
    print("Running Hierarchical Clustering Optimization with Network Constraints")
    port = rp.HCPortfolio(returns=returns)
    w = port.optimization(model='HERC', codependence='pearson', linkage='ward', risk_measure='MV')
    return w

# 10. Real-World Application with MOSEK
def mosek_real_world_optimization(returns):
    print("Running Optimization with MOSEK Solver")
    # MOSEK integration example
    # port.optimization(model='Classic', solver='MOSEK')  # Uncomment if MOSEK is available
    pass

# Main execution flow
if __name__ == "__main__":
    # Example usage for testing (replace with your actual data)

    # Historical Mean-Risk Optimization
    w_mean_risk = mean_risk_optimization(data)

    # Custom Mean-Risk Optimization (Example with random custom estimates)
    #custom_mean = pd.Series(np.random.rand(data.shape[1]), index=data.columns)
    #custom_cov = pd.DataFrame(np.random.rand(data.shape[1], data.shape[1]), index=data.columns, columns=data.columns)
    #w_custom = mean_risk_custom_optimization(data, custom_mean, custom_cov)

    # Ulcer Index Optimization
    w_ui = ulcer_index_optimization(data)

    # Entropic Value at Risk Optimization
    w_evar = evar_optimization(data)

    # Gini Mean Difference Optimization
    w_gmd = gmd_optimization(data)

    # Black-Litterman Example (Dummy inputs for P, Q, Omega)
    P = pd.DataFrame(np.random.rand(2, data.shape[1]), columns=data.columns)
    Q = pd.Series(np.random.rand(2))
    Omega = np.diag(np.random.rand(2))
    w_bl = black_litterman_optimization(data, P, Q, Omega)

    # Risk Parity Optimization
    w_rp = risk_parity_optimization(data)

    # Hierarchical Risk Parity Optimization
    w_hrp = hrp_optimization(data)

    # Hierarchical Clustering Optimization with Network Constraints
    w_herc = hierarchical_clustering_optimization(data)

    # Real-World Application with MOSEK
    mosek_real_world_optimization(data)

    # Print results (for each optimization run)
    print("Mean-Risk Weights: ", w_mean_risk)
    print("Custom Mean-Risk Weights: ", w_custom)
    print("Ulcer Index Weights: ", w_ui)
    print("EVaR Weights: ", w_evar)
    print("GMD Weights: ", w_gmd)
    print("Black-Litterman Weights: ", w_bl)
    print("Risk Parity Weights: ", w_rp)
    print("HRP Weights: ", w_hrp)
    print("HERC Weights: ", w_herc)


Running Mean-Risk Portfolio Optimization using historical estimates
Running Ulcer Index Portfolio Optimization


NameError: name 'optimize_portfolio' is not defined